In [ ]:
!pip install mido
!pip install pygame
!pip install music21

In [ ]:
import pandas as pd
import numpy as np
from mido import MidiFile
import pygame
import IPython
import matplotlib.pyplot as plt
import librosa.display
import keras.layers as L
import keras.models as M
import keras
from keras.layers import SimpleRNN,LSTM,GRU
from sklearn.model_selection import train_test_split
from IPython import *
from music21 import *
import os
import tensorflow as tf

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
data=pd.read_csv('../input/musicnet-dataset/musicnet_metadata.csv')

In [ ]:
# Making live plots
# Importing the mid file first
mid=MidiFile('../input/musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven/2313_qt15_1.mid',clip=True)
# Having a look at the tracks of the mid file
mid.tracks

In [ ]:
# Having a look at the msges of the tracks let's have a look at the first one since it only have 325 messages
for i in mid.tracks[1] :
    print(i)

In [ ]:
# Well you can see clearly there are many meta messages here how will we solve that ??
# I got an idea let's get all the values with note on here 
note_on=[]
n=50
for m in range(n):
    mid=MidiFile('../input/musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven/'+os.listdir('../input/musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven')[m],clip=True)
    for j in range(len(mid.tracks)):
        for i in mid.tracks[j] :
            if str(type(i))!="<class 'mido.midifiles.meta.MetaMessage'>" :
                x=str(i).split(' ')
                if x[0]=='note_on':
                    note_on.append(int(x[2].split('=')[1]))

In [ ]:
import time
from IPython.display import clear_output
def live_plot_make(x,range_=20,pause_time=0.01,skip_a_do=1):
    for i in range(0,len(x)-range_,skip_a_do):
        plt.figure(figsize=(18,8))
        x_plot=x[i:i+range_]
        y_plot=[i for i in range(range_)]
        fig=plt.plot(y_plot,x_plot,marker='D')
        plt.ylim([min(x),max(x)])
        time.sleep(pause_time)
        clear_output(wait=True)
        plt.show()


In [ ]:
live_plot_make(note_on,range_=100,pause_time=0.02,skip_a_do=1000)

## Hehe something i created from scratch :) Hope you like this animated plot :)
## You might need to save this to gif if you wanna add it in markdown

# First Step Creating Data For Our Model

In [ ]:
len(note_on)

In [ ]:
'''We are gonna take a frame of 10 notes at a time since we have 1437 notes takin 10 notes at a time will give us
        about 1427 training samples . Lets try :)'''
training_data=[]
labels=[]
for i in range(20,len(note_on)):
    training_data.append(note_on[i-20:i])
    labels.append(note_on[i])

In [ ]:
print('Training data is :',training_data[0],' The label for it is :',labels[0])

In [ ]:
# How many different type of notes can we have in the labels column
different_labels=set(labels)
print('The different type of labels that we can have and might need to predict are : ',len(different_labels))

In [ ]:
# Function to build the model
def build_model():
    model=M.Sequential()
    model.add(LSTM(128,input_shape=(10,1)))
    model.add(L.Dense(1,'relu'))
    model.compile(loss='MSE',optimizer='adam')
    return model

In [ ]:
def build_model_softmax():
    model=M.Sequential()
    model.add(LSTM(128,input_shape=(10,1),return_sequences=True))
    model.add(LSTM(128))
    model.add(L.Flatten())
    model.add(L.Dense(40,'relu'))
    model.add(L.Dense(40,'softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam')
    return model

In [ ]:
def build_model_lstm2():
    model=M.Sequential()
    model.add(LSTM(200,input_shape=(10,1),unroll=True,return_sequences=True))
    model.add(L.Dropout(0.4))
    model.add(LSTM(100))
    model.add(L.Dense(100,'relu'))
    model.add(L.Dropout(0.2))
    model.add(L.Dense(1,'relu'))
    model.compile(loss='MSE',optimizer='adam')
    return model

In [ ]:
def build_model_softmax():
    model=M.Sequential()
    model.add(LSTM(200,input_shape=(10,1),unroll=True,return_sequences=True))
    model.add(L.Dropout(0.4))
    model.add(LSTM(100))
    model.add(L.Dense(100,'relu'))
    model.add(L.Dropout(0.2))
    model.add(L.Dense(1,'relu'))
    model.compile(loss='MSE',optimizer='adam')
    return model

In [ ]:
with strategy.scope():
    model=build_model_lstm2()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0)

In [ ]:
training_data=np.array(training_data)
training_data=training_data.reshape((training_data.shape[0],training_data.shape[1],1))
labels=np.array(labels)

In [ ]:
# Let's make training and validation data
X_train, X_test, y_train, y_test = train_test_split(training_data, labels, test_size=0.05, random_state=42)

In [ ]:
X_train.shape

In [ ]:
model.fit(X_train,y_train,epochs=200,batch_size=32 * strategy.num_replicas_in_sync,
          validation_data=(X_test,y_test),callbacks=[early_stop])

In [ ]:
# Saving the model
model.save('Auto_Song_Maker.h5')

In [ ]:
history=model.history.history
plt.plot([i for i in range(len(history['loss']))],history['loss'])
plt.plot([i for i in range(len(history['val_loss']))],history['val_loss'])

In [ ]:
training_data[0].reshape(1,20,1)

In [ ]:
model.predict(training_data[0].reshape(1,20,1))[0][0]

In [ ]:
###### Let's do some random predictions now
n=200
starter_notes=training_data[0]
x=training_data[0].reshape(1,20,1)
tune=list(training_data[0].reshape(-1,))
for i in range(n) :
    pred=int(model.predict(x)[0][0])
    if round(pred)==round(tune[-1]):
        p=np.random.choice(['a','b','c'])
        if p=='a':
            pred=65
        elif p=='b':
            pred=60
        else:
            pred=70
    tune.append(pred)
    x=tune[-10:]
    x=np.array(x)
    x=x.reshape(1,10,1)
    

In [ ]:
tune=list(np.array(tune).astype('float32'))

# That was just one song :) Don't know how much chaos will occur after n songs :)

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for patterns in tune:
    pattern=str(patterns)
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
output_notes

In [ ]:
midi_stream = stream.Stream(output_notes)


In [ ]:
midi_stream.write('midi','test_output.mid')

## You might need to wait until the popup window on the top right dissappears

In [ ]:
IPython.display.Audio('./test_output.wav')

# Kinda weird tune but it will learn in time :)

# Thank you  Hope you liked it :)

# Try running the file . The plot above is animation but will only work while file is running :)